In [2]:
import os 
import importlib
import textwrap
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, get_response_synthesizer, PromptHelper
from llama_index.text_splitter import SentenceSplitter
from llama_index.node_parser import SimpleNodeParser
from llama_index.vector_stores import ChromaVectorStore
from llamaindex_object_array_reader.dataset import simple_ols # import a simple dataset 
from llama_index.llms import HuggingFaceLLM
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import BitsAndBytesConfig
from llama_index.llms import Ollama
from llama_index import ServiceContext, set_global_tokenizer
# from langchain.embeddings import HuggingFaceEmbedding, HuggingFaceInstructEmbeddings
from llama_index.embeddings import HuggingFaceEmbedding
from transformers import AutoTokenizer, AutoModel
from argparse import Namespace
from chromadb import Collection, PersistentClient
from dotenv import load_dotenv
from llamaindex_object_array_reader import ObjectArrayReader
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
import nest_asyncio

# 允许嵌套事件循环
nest_asyncio.apply()



ImportError: cannot import name 'VectorStoreIndex' from 'llama_index' (unknown location)

In [2]:
import logging
import sys
from llamaindex_object_array_reader._logging import logger

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
log = logger
log.setLevel(logging.DEBUG)

In [3]:
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [4]:
# Obsolete
# if os.path.exists('my_cred.py'):
#     my_cred = importlib.import_module('my_cred')
#     os.environ['OPENAI_API_KEY'] = my_cred.OPENAI_API_KEY
# else:
#     # Set your OPENAI API Key
#     os.environ['OPENAI_API_KEY'] = "vy-...cH5N"

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
HF_TOKEN = os.environ['HF_TOKEN']

In [5]:
def print_resp(msg, max_len:int=55):
    """将文本分割为每行最大长度的子字符串
    """
    divider: str = '\n'+ '*'*60+'\n'
    msg = textwrap.fill(msg, width=max_len)
    print(f"""\u2705 RESPONSE:{divider}\n{msg}\n{divider} \U0001F6A9END OF RESPONSE""")

In [6]:
models:Namespace = Namespace(
    BERT_BASE_CHINESE="bert-base-chinese",
    LLAMA2_CHINESE_7B_CHAT="FlagAlpha/Llama2-Chinese-7b-Chat", #18G needed
    LLAMA2_7B_CHAT_HF="meta-llama/Llama-2-7b-chat-hf", #18G needed
    BLOOM_560M="bigscience/bloom-560m", #18G needed
    BLOOMZ_560M="bigscience/bloomz-560m", #18G needed
    GPT2="GPT2", #18G needed
    ALL_MPNET_BASE_V2="sentence-transformers/all-mpnet-base-v2", #18G needed
    MISTRAL_7B_INSTRUCT_V0_1="mistralai/Mistral-7B-Instruct-v0.1", #18G needed
    STARLING_LM_7B="berkeley-nest/Starling-LM-7B-alpha",
)

In [7]:
# Set the check point
check_point:str = models.ALL_MPNET_BASE_V2

In [8]:
tokenizer = AutoTokenizer.from_pretrained(check_point)
set_global_tokenizer(tokenizer)

# Alternatively, using a local LLM
USE_LOCAL:bool = True
if USE_LOCAL:
    # llm = Ollama(model="llama2-chinese")
    # llm = Ollama(model="starling-lm:7b-alpha-q3_K_M")
    llm = Ollama(model="mistral")
    
else: 
    llm = HuggingFaceLLM(
        model_name=check_point,
        tokenizer_name=check_point,
        context_window=512,
        model_kwargs={
            # 'torch_dtype':torch.float16,
            "token": HF_TOKEN,
            'load_in_8bit':False, #No, the bitsandbytes library only works on CUDA GPU. So it must set to 'False' as running on mac os. 
            'offload_folder':"offload_folder",
            'offload_state_dict':True,
            'is_decoder': True if check_point==models.BERT_BASE_CHINESE else None,
            },
        tokenizer_kwargs={
            "token": HF_TOKEN,
            "return_tensors":'pt',},
        device_map="auto" if check_point!=models.BERT_BASE_CHINESE else "mps", 
    )


Starting new HTTPS connection (1): huggingface.co:443


https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


In [9]:
embedding_model = HuggingFaceEmbedding(
    model_name=check_point,
    tokenizer=tokenizer,
    cache_folder="cache_folder",
    max_length=512,
    device="mps"
)

https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/config.json HTTP/1.1" 200 0


In [10]:
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=64)
prompt_helper = PromptHelper(
    context_window=512,
    num_output=256,
    chunk_overlap_ratio=0.1,
    chunk_size_limit=None,
)

In [11]:
documents = SimpleDirectoryReader("test_docs/simple_txt_short_en").load_data()

> [SimpleDirectoryReader] Total files added: 3


In [12]:
# Assuming documents have already been loaded
# Initialize the parser
parser = SimpleNodeParser.from_defaults(chunk_size=512, chunk_overlap=20)
# Parse documents into nodes
nodes = parser.get_nodes_from_documents(documents)
print('Total nodes:', len(nodes))
for _, n in enumerate(nodes):
    print(n)
    print('---')

> Adding chunk: You can do data integration, management, analys...
> Adding chunk: Colosscious' flagship product, Pharmquer, is an...
> Adding chunk: Welcome to Colosscious. 
We are the expert who ...
Total nodes: 3
Node ID: 1b90da42-2951-4f96-b2a8-6ee9547b1618
Text: You can do data integration, management, analysis and composing
reports and dashboards with Pharmquer, and then automatize all your
works.
---
Node ID: 038bfbf8-77c4-4e82-9bf9-1706442077e2
Text: Colosscious' flagship product, Pharmquer, is an enterprise level
software of manufacturing and business intelligence, which is
architected especially for the industry.
---
Node ID: 49a00622-84ed-4f52-89c5-f6a4856cdea5
Text: Welcome to Colosscious.  We are the expert who spotlight-focus
on providing the digital technology to bio and pharmaceutical
companies, engaging in boosting the performances of new drug
developments, quality control, manufacturing processes, and reducing
the costs and duration by Big Data.
---


In [13]:
V_DB_NAME = "chromadb"
chroma_client = PersistentClient(V_DB_NAME)
COLLECTION_NAME:str = 'test'
chroma_collection:Collection = chroma_client.get_or_create_collection(COLLECTION_NAME)
vector_store = ChromaVectorStore(chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


2024-02-12 22:48:51,212 - chromadb.telemetry.product.posthog - INFO - (posthog.py:20) - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information. 


Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Starting component System
Starting component Posthog
Starting component OpenTelemetryClient
Starting component SimpleAssignmentPolicy
Starting component SqliteDB
Starting component LocalSegmentManager
Starting component SegmentAPI


In [14]:
for n in nodes:
    print(storage_context.docstore.document_exists(n.id_))

False
False
False


Starting new HTTPS connection (1): app.posthog.com:443
https://app.posthog.com:443 "POST /batch/ HTTP/1.1" 200 None


## Create and store new embeddings to ChromaDB. 

In [ ]:
storage_context.docstore.add_documents(nodes)

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding_model, text_splitter=text_splitter,
    prompt_helper=prompt_helper, callback_manager=callback_manager)
# index = VectorStoreIndex.from_documents(
#     documents, service_context=service_context, storage_context=storage_context, show_progress=True,
# )
index = VectorStoreIndex(
    nodes, service_context=service_context, storage_context=storage_context, show_progress=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Generating embeddings: 100%|██████████| 3/3 [00:00<00:00,  5.29it/s]
2024-02-07 22:47:49,898 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35 


Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35


2024-02-07 22:47:49,898 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862 


Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862


2024-02-07 22:47:49,899 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: cb553733-838a-421b-89bf-c582fe90182a 


Add of existing embedding ID: cb553733-838a-421b-89bf-c582fe90182a


In [ ]:
# example: 
# "GPT4 Correct User: {prompt}<|end_of_turn|>GPT4 Correct Assistant: {response}<|end_of_turn|>GPT4 Correct User: {follow_up_question}<|end_of_turn|>GPT4 Correct Assistant:"
# ref: https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha
sep = '<|end_of_turn|>'
resp_prompt_temp = "GPT4 Correct Assistant: "

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
tokenizer(
    ["What Colosscious do?"],
    return_tensors="pt",
    add_special_tokens=False,
).input_ids.to("mps")

tensor([[ 2058,  8906, 15098, 18440,  2083,  1033]], device='mps:0')

In [ ]:
query_resp = query_engine.query("What is flagship product of Colosscious")

print_resp(query_resp.response)

2024-02-07 19:52:25,347 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

Sorry, I cannot answer your query without using any
more tools.

************************************************************
 🚩END OF RESPONSE


In [ ]:
query_engine = index.as_chat_engine()
query_resp = query_engine.query("What is Pharmquer?")
print_resp(query_resp.response)

2024-02-07 19:52:36,318 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

PharmQuer is an international pharmacovigilance
electronic system used in more than 80 countries for
the collection and analysis of spontaneous case reports
(adverse reactions to drugs). It is a free, web-based
platform that allows users to report, review and
analyze cases. The primary purpose of PharmQuer is to
facilitate data sharing between regulatory agencies,
pharmaceutical companies, academia, and other
stakeholders in the field of pharmacovigilance.

************************************************************
 🚩END OF RESPONSE


## Load existing embeddings in ChromaDB.

In [15]:
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding_model, text_splitter=text_splitter,
    prompt_helper=prompt_helper, callback_manager=callback_manager)
# load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context, service_context=service_context
)


**********
Trace: index_construction
**********


In [16]:
# create a query engine
query_engine = index.as_query_engine()

In [17]:
response = query_engine.query("What is Colosscious?")
print_resp(response.response)

Starting component PersistentLocalHnswSegment


2024-02-12 18:47:29,558 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35 


Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35


2024-02-12 18:47:29,560 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862 


Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862


...

HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
> Refine context: and reducing the costs and duration by Big Data...
load_ssl_context verify=True cert=None trust_env=True http2=False
load_verify_locations cafile='/Users/yuwang/Developments/python/llamaindex_object_array_reader/.venv/lib/python3.10/site-packages/certifi/cacert.pem'
connect_tcp.started host='localhost' port=11434 local_address=None timeout=30.0 socket_options=None
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2bc062e00>
send_request_headers.started request=<Request [b'POST']>
send_request_headers.complete
send_request_body.started request=<Request [b'POST']>
send_request_body.complete
receive_response_headers.started request=<Request [b'POST']>
receive_response_headers.complete return_value=(b'HTTP/1.

2024-02-12 18:47:39,390 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
> Refine context: new drug developments, quality control, manufac...
load_ssl_context verify=True cert=None trust_env=True http2=False
load_verify_locations cafile='/Users/yuwang/Developments/python/llamaindex_object_array_reader/.venv/lib/python3.10/site-packages/certifi/cacert.pem'
connect_tcp.started host='localhost' port=11434 local_address=None timeout=30.0 socket_options=None
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2c5ea00d0>
send_request_headers.started request=<Request [b'POST']>
send_request_headers.complete
send_request_body.started request=<Request [b'POST']>
send_request_body.complete
receive_response_headers.started request=<Request [b'POST']>
receive_response_headers.complete return_value=(b'HTTP/1.

2024-02-12 18:47:41,471 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
**********
Trace: query
    |_query ->  12.225892 seconds
      |_retrieve ->  0.321584 seconds
        |_embedding ->  0.278516 seconds
      |_synthesize ->  11.90415 seconds
        |_templating ->  2.2e-05 seconds
        |_llm ->  7.398382 seconds
        |_templating ->  3.7e-05 seconds
        |_llm ->  2.408754 seconds
        |_templating ->  7.5e-05 seconds
        |_llm ->  2.073766 seconds
**********
✅ RESPONSE:
************************************************************

 Coloscius is an organization that provides advanced
digital technology solutions for the bio and
pharmaceutical sectors. Leveraging Big Data, they
strive to optimize new drug development, ensure
superior quality control, improve manufacturing
workflows, and reduce associat

In [ ]:
response = query_engine.query("What is Colosscious?")
print_resp(response.response)

2024-02-11 22:42:31,867 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35 


Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35


2024-02-11 22:42:31,867 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862 


Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862


2024-02-11 22:42:31,868 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: cb553733-838a-421b-89bf-c582fe90182a 


...

HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


2024-02-11 22:42:42,201 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


2024-02-11 22:42:44,876 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

 Colosscious is a platform that focuses on delivering
digital technology solutions to bio and pharmaceutical
companies, with the goal of optimizing new drug
developments, ensuring quality control, streamlining
manufacturing processes, and reducing costs and
duration through the use of advanced data analysis
techniques.

************************************************************
 🚩END OF RESPONSE


## Use llama_index_object_array_reader

In [17]:
# Preview: demo data
simple_ols[:2]

[{'x1': 97.98219999874924,
  'x2': 99.84941752810117,
  'x3': 100.9727776594234,
  'y': 360.87650920565545},
 {'x1': 101.00077953260389,
  'x2': 99.87874921228179,
  'x3': 99.35642250227457,
  'y': 361.50488035486944}]

In [18]:
loader = ObjectArrayReader()

In [19]:
from llama_index.readers.schema.base import Document
object_arrays:list[Document] = loader.load_data(file=simple_ols)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      50 non-null     float64
 1   x2      50 non-null     float64
 2   x3      50 non-null     float64
 3   y       50 non-null     float64
dtypes: float64(4)
memory usage: 1.7 KB


In [20]:
import pandas as pd
df = pd.DataFrame(simple_ols)

In [21]:
object_arrays[0]

Document(id_='24cb5c07-8129-48a6-b1a5-114274000fe1', embedding=None, metadata={'columns': "['x1', 'x2', 'x3', 'y']", 'schema': 'None', 'shape': '(50, 4)'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='01044d00146a997fca8953cf8ba579cb4492a76d451ca3aa31645e0e2e9bcb89', text='97.98219999874924, 99.84941752810117, 100.9727776594234, 360.87650920565545\n101.00077953260389, 99.87874921228179, 99.35642250227457, 361.50488035486944\n98.5109626677227, 100.7485502397903, 99.46465098250788, 359.8117609861218\n100.77335929310553, 100.03722922045552, 99.86657209922947, 362.2336960397953\n100.97359840386007, 99.1724799721807, 100.16093297144785, 362.1391160315852\n100.18799255929102, 100.55900119891184, 100.61532849440285, 363.29752977180965\n100.9157547652626, 98.61649241995889, 99.06726035297895, 359.7975894964005\n101.04615952660859, 102.00920930524853, 100.16419028246959, 364.8003752715575\n99.46321248760913, 100.23898461781165, 100.4603474082993, 361.9

In [22]:
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding_model, callback_manager=callback_manager)
index = VectorStoreIndex.from_documents(
    documents=object_arrays, service_context=service_context,  storage_context=storage_context, show_progress=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (2225 > 512). Running this sequence through the model will result in indexing errors


> Adding chunk: 97.98219999874924, 99.84941752810117, 100.97277...
> Adding chunk: 98.90754655604644, 99.80434032022505, 360.25326...
> Adding chunk: 26476906248293, 100.78711425646654, 100.2982222...
> Adding chunk: 100.84958819151115, 361.7497627252443
99.070933...


Generating embeddings: 100%|██████████| 4/4 [00:00<00:00,  6.01it/s]

Starting component PersistentLocalHnswSegment



2024-02-12 18:57:31,224 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35 


...

2024-02-12 18:57:31,225 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862 


Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862


2024-02-12 18:57:31,226 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: cb553733-838a-421b-89bf-c582fe90182a 


Add of existing embedding ID: cb553733-838a-421b-89bf-c582fe90182a
**********
Trace: index_construction
    |_node_parsing ->  0.05153 seconds
      |_chunking ->  0.022632 seconds
    |_embedding ->  0.664412 seconds
**********


https://app.posthog.com:443 "POST /batch/ HTTP/1.1" 200 None


In [21]:
len(simple_ols)

50

In [22]:
object_arrays

[Document(id_='7e10f455-a8eb-49c9-999b-2e4f74124067', embedding=None, metadata={'columns': "['x1', 'x2', 'x3', 'y']", 'schema': 'None', 'shape': '(50, 4)'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='01044d00146a997fca8953cf8ba579cb4492a76d451ca3aa31645e0e2e9bcb89', text='97.98219999874924, 99.84941752810117, 100.9727776594234, 360.87650920565545\n101.00077953260389, 99.87874921228179, 99.35642250227457, 361.50488035486944\n98.5109626677227, 100.7485502397903, 99.46465098250788, 359.8117609861218\n100.77335929310553, 100.03722922045552, 99.86657209922947, 362.2336960397953\n100.97359840386007, 99.1724799721807, 100.16093297144785, 362.1391160315852\n100.18799255929102, 100.55900119891184, 100.61532849440285, 363.29752977180965\n100.9157547652626, 98.61649241995889, 99.06726035297895, 359.7975894964005\n101.04615952660859, 102.00920930524853, 100.16419028246959, 364.8003752715575\n99.46321248760913, 100.23898461781165, 100.4603474082993, 361.

In [22]:
# create a query engine
query_engine = index.as_query_engine()

In [19]:
response = query_engine.query("How many values with in the dataset?")
print_resp(response.response)

Starting component PersistentLocalHnswSegment


2024-02-12 17:22:49,990 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35 


Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35


2024-02-12 17:22:49,990 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862 


Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862


...

HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
> Refine context: 98.52851110733191, 360.68518093107895
101.14589...
load_ssl_context verify=True cert=None trust_env=True http2=False
load_verify_locations cafile='/Users/yuwang/Developments/python/llamaindex_object_array_reader/.venv/lib/python3.10/site-packages/certifi/cacert.pem'
connect_tcp.started host='localhost' port=11434 local_address=None timeout=30.0 socket_options=None
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2ca8a4e50>
send_request_headers.started request=<Request [b'POST']>
send_request_headers.complete
send_request_body.started request=<Request [b'POST']>
send_request_body.complete
receive_response_headers.started request=<Request [b'POST']>
receive_response_headers.complete return_value=(b'HTTP/1.

2024-02-12 17:23:19,693 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
> Refine context: 99.21840952279409, 98.11881715092599, 358.86788...
load_ssl_context verify=True cert=None trust_env=True http2=False
load_verify_locations cafile='/Users/yuwang/Developments/python/llamaindex_object_array_reader/.venv/lib/python3.10/site-packages/certifi/cacert.pem'
connect_tcp.started host='localhost' port=11434 local_address=None timeout=30.0 socket_options=None
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2ca8a4430>
send_request_headers.started request=<Request [b'POST']>
send_request_headers.complete
send_request_body.started request=<Request [b'POST']>
send_request_body.complete
receive_response_headers.started request=<Request [b'POST']>
receive_response_headers.complete return_value=(b'HTTP/1.

2024-02-12 17:23:21,622 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
**********
Trace: query
    |_query ->  32.061951 seconds
      |_retrieve ->  0.432426 seconds
        |_embedding ->  0.404898 seconds
      |_synthesize ->  31.629347 seconds
        |_templating ->  2.4e-05 seconds
        |_llm ->  9.342469 seconds
        |_templating ->  3.8e-05 seconds
        |_llm ->  2.498338 seconds
        |_templating ->  3.2e-05 seconds
        |_llm ->  8.683066 seconds
        |_templating ->  1.7e-05 seconds
        |_llm ->  1.25369 seconds
        |_templating ->  2.8e-05 seconds
        |_llm ->  2.081659 seconds
        |_templating ->  2.9e-05 seconds
        |_llm ->  1.898468 seconds
        |_templating ->  2.7e-05 seconds
        |_llm ->  1.834185 seconds
        |_templating ->  3.7e-05 seconds
        |_llm 

In [ ]:
response = query_engine.query("How many values with in the dataset?")
print_resp(response.response)

2024-02-11 22:43:12,952 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

 There are a total of 50 rows (or samples) in the
provided dataset, as indicated by the shape attribute
which is (50, 4). Each row represents one sample and
contains four features: 'x1', 'x2', 'x3', and 'y'.

************************************************************
 🚩END OF RESPONSE


In [26]:
df.shape

(50, 4)

In [29]:
response = query_engine.query("How many columns' name starts with 'x'?")
print_resp(response.response)

2024-02-08 15:13:24,979 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

 There are three columns' names that start with 'x',
which are 'x1', 'x2', and 'x3'.

************************************************************
 🚩END OF RESPONSE


In [24]:
response = query_engine.query("What is the average of column 'x1'?")
print_resp(response.response)

2024-02-08 15:21:03,508 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

 To find the average of column 'x1', we need to sum all
the values in column 'x1' and then divide by the total
number of rows (50). Here are all the values in column
'x1':  99.07093312420517, 100.84958819151115,
99.72656237514776, ... , 100.97373793175905  Adding all
the values gives us:  99.07093312420517 +
100.84958819151115 + ... + 100.97373793175905 = (sum of
all x1 values)  Now, we need to divide the sum by the
total number of rows, which is 50:  (sum of all x1
values) / 50 = average of column 'x1'  Without
calculating the exact sum, we can see that the average
value lies between 98.73 (lowest value) and 100.97
(highest value). However, without performing the actual
calculation, we cannot provide an exact numerical
answer for the average of column 'x1'.  Please note
that providing the exact average would require further
calculations that go

In [27]:
df.head()

,x1,x2,x3,y
0,97.982200,99.849418,100.972778,360.876509
1,101.000780,99.878749,99.356423,361.504880
2,98.510963,100.748550,99.464651,359.811761
3,100.773359,100.037229,99.866572,362.233696
4,100.973598,99.172480,100.160933,362.139116


In [35]:
print(df['x1'][:5])
print('Mean= ', df['x1'].mean())

0     97.982200
1    101.000780
2     98.510963
3    100.773359
4    100.973598
Name: x1, dtype: float64
Mean=  100.07520939069373


### Sub Question Query Engine

In [17]:
query_engine = index.as_query_engine()

In [16]:
response = query_engine.query(
    "What about the dataset?"
)
print_resp(response.response)

Starting component PersistentLocalHnswSegment


2024-02-12 19:03:13,439 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35 


Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35


2024-02-12 19:03:13,440 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862 


Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862


...

HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
load_ssl_context verify=True cert=None trust_env=True http2=False
load_verify_locations cafile='/Users/yuwang/Developments/python/llamaindex_object_array_reader/.venv/lib/python3.10/site-packages/certifi/cacert.pem'
connect_tcp.started host='localhost' port=11434 local_address=None timeout=30.0 socket_options=None
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2bd9c6920>
send_request_headers.started request=<Request [b'POST']>
send_request_headers.complete
send_request_body.started request=<Request [b'POST']>
send_request_body.complete
receive_response_headers.started request=<Request [b'POST']>
receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'

2024-02-12 19:06:28,180 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
> Refine context: 100.72347518969544, 99.54248360001975, 362.1927...
load_ssl_context verify=True cert=None trust_env=True http2=False
load_verify_locations cafile='/Users/yuwang/Developments/python/llamaindex_object_array_reader/.venv/lib/python3.10/site-packages/certifi/cacert.pem'
connect_tcp.started host='localhost' port=11434 local_address=None timeout=30.0 socket_options=None
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2bd9c54b0>
send_request_headers.started request=<Request [b'POST']>
send_request_headers.complete
send_request_body.started request=<Request [b'POST']>
send_request_body.complete
receive_response_headers.started request=<Request [b'POST']>
receive_response_headers.complete return_value=(b'HTTP/1.

2024-02-12 19:06:34,540 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
**********
Trace: query
    |_query ->  201.39414 seconds
      |_retrieve ->  0.294847 seconds
        |_embedding ->  0.268448 seconds
      |_synthesize ->  201.099127 seconds
        |_templating ->  2.3e-05 seconds
        |_llm ->  5.827234 seconds
        |_templating ->  3.1e-05 seconds
        |_llm ->  2.544976 seconds
        |_templating ->  1.2e-05 seconds
        |_llm ->  3.841746 seconds
        |_templating ->  3.3e-05 seconds
        |_llm ->  4.65833 seconds
        |_templating ->  1.6e-05 seconds
        |_llm ->  5.651133 seconds
        |_templating ->  1.7e-05 seconds
        |_llm ->  4.988026 seconds
        |_templating ->  2.7e-05 seconds
        |_llm ->  4.585943 seconds
        |_templating ->  1.1e-05 seconds
        |_llm

In [18]:
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="summary_tool",
            description=f"Return the shape of the dataset and the basic summary of the dataset, such as mean, range, stddev of each columns.",
        ),
    ),
] 

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    verbose=True,
    use_async=True,
)

In [19]:
response = query_engine.query(
   "What about the dataset?"
)
print_resp(response.response )

load_ssl_context verify=True cert=None trust_env=True http2=False
load_verify_locations cafile='/Users/yuwang/Developments/python/llamaindex_object_array_reader/.venv/lib/python3.10/site-packages/certifi/cacert.pem'
connect_tcp.started host='localhost' port=11434 local_address=None timeout=30.0 socket_options=None
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x29fc83790>
send_request_headers.started request=<Request [b'POST']>
send_request_headers.complete
send_request_body.started request=<Request [b'POST']>
send_request_body.complete
receive_response_headers.started request=<Request [b'POST']>
receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Mon, 12 Feb 2024 21:49:18 GMT'), (b'Content-Length', b'897')])


2024-02-12 22:49:18,098 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
檢查 {'items': [{'sub_question': 'What is the shape of the dataset', 'tool_name': 'summary_tool'}, {'sub_question': 'What is the mean of each column in the dataset', 'tool_name': 'summary_tool'}, {'sub_question': 'What is the range of each column in the dataset', 'tool_name': 'summary_tool'}, {'sub_question': 'What is the standard deviation of each column in the dataset', 'tool_name': 'summary_tool'}]}
Generated 4 sub questions.
[summary_tool] Q: What is the shape of the dataset
Starting component PersistentLocalHnswSegment


2024-02-12 22:49:18,860 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35 


Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35


...

HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
> Refine context: 99.7129946461946, 101.44068386026443, 363.74407...


2024-02-12 22:50:33,443 - llama_index.query_engine.sub_question_query_engine - WARNING - (sub_question_query_engine.py:237) - [summary_tool] Failed to run What is the standard deviation of each column in the dataset 


[summary_tool] Failed to run What is the standard deviation of each column in the dataset
load_ssl_context verify=True cert=None trust_env=True http2=False
load_verify_locations cafile='/Users/yuwang/Developments/python/llamaindex_object_array_reader/.venv/lib/python3.10/site-packages/certifi/cacert.pem'
connect_tcp.started host='localhost' port=11434 local_address=None timeout=30.0 socket_options=None
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2c25bf2b0>
send_request_headers.started request=<Request [b'POST']>
send_request_headers.complete
send_request_body.started request=<Request [b'POST']>
send_request_body.complete
receive_response_headers.started request=<Request [b'POST']>
receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Mon, 12 Feb 2024 21:50:35 GMT'), (b'Content-Length', b'545')])


2024-02-12 22:50:35,463 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
receive_response_body.started request=<Request [b'POST']>
receive_response_body.complete
response_closed.started
response_closed.complete
close.started
close.complete
**********
Trace: query
    |_query ->  88.110441 seconds
      |_templating ->  1.4e-05 seconds
      |_llm ->  10.748315 seconds
      |_sub_question ->  34.018639 seconds
        |_query ->  34.018035 seconds
          |_retrieve ->  0.756852 seconds
            |_embedding ->  0.705106 seconds
          |_synthesize ->  33.260937 seconds
            |_templating ->  2.4e-05 seconds
            |_llm ->  1.93535 seconds
            |_llm ->  1.935253 seconds
            |_templating ->  2.5e-05 seconds
            |_llm ->  3.632835 seconds
            |_llm ->  3.632734 seconds
            |_templating ->  1.8e-05 seconds
            |_llm ->  2.892591 seconds
            |_llm ->  2.892515 seconds
            |_templating ->  2.3e-05 seconds
       

In [38]:
# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.callbacks.schema import CBEventType, EventPayload

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    print(start_event)
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")